In [1]:
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews, app
from flair.models import TARSClassifier
from flair.data import Sentence
import nltk
import time
from nltk import tokenize
import spacy
import re
import requests
import json

In [2]:
ford_app_img = 'com.ford.fordpassimg' #com.ford.fordpass
ford_app_us ='com.ford.fordpass'
ford_app_eu = 'com.ford.fordpasseu'
ford_app_ap = 'com.ford.fordpassap'
ford_app_za ='com.ford.fordpassza'
ford_app_img_reviews = []
ford_app_us_reviews = []
ford_app_eu_reviews = []
ford_app_ap_reviews = []

In [3]:
#fetch the reviews from play store using 'reviews' method
img_reviews, token1 = reviews(
    ford_app_img,
    country = 'za',
    sort = Sort.NEWEST,
    count = 500,
    filter_score_with = None
)

us_reviews, token2 = reviews(
    ford_app_us,
    country = 'us',
    sort = Sort.NEWEST,
    count = 500,
    filter_score_with = None
)

eu_reviews, token3 = reviews(
    ford_app_eu,
    country = 'uk',
    sort = Sort.NEWEST,
    count = 500,
    filter_score_with = None
)

ap_reviews, token4 = reviews(
    ford_app_img,
    country = 'au',
    sort = Sort.NEWEST,
    count = 500,
    filter_score_with = None
)

In [4]:
for img_review in img_reviews:
    review = {}
    review['Name'] = img_review["userName"]
    review['Rating'] = img_review["score"]
    review['Review'] = img_review["content"]
    review['Date'] = img_review['at']
    review['Country'] = "ZA"
    review['Version'] = img_review['reviewCreatedVersion']
    ford_app_img_reviews.append(review)
img_reviews_df = pd.DataFrame(ford_app_img_reviews)

#US
for us_review in us_reviews:
    review = {}
    review['Name'] = us_review["userName"]
    review['Rating'] = us_review["score"]
    review['Review'] = us_review["content"]
    review['Date'] = us_review['at']
    review['Country'] = "NA"
    review['Version'] = us_review['reviewCreatedVersion']
    ford_app_us_reviews.append(review)
us_reviews_df = pd.DataFrame(ford_app_us_reviews)

#EU
for eu_review in eu_reviews:
    review = {}
    review['Name'] = eu_review["userName"]
    review['Rating'] = eu_review["score"]
    review['Review'] = eu_review["content"]
    review['Date'] = eu_review['at']
    review['Country'] = "UK"
    review['Version'] = eu_review['reviewCreatedVersion']
    ford_app_eu_reviews.append(review)
eu_reviews_df = pd.DataFrame(ford_app_eu_reviews)

#AP
for ap_review in ap_reviews:
    review = {}
    review['Name'] = ap_review["userName"]
    review['Rating'] = ap_review["score"]
    review['Review'] = ap_review["content"]
    review['Date'] = ap_review['at']
    review['Country'] = "AU/NZ"
    review['Version'] = ap_review['reviewCreatedVersion']
    ford_app_ap_reviews.append(review)
ap_reviews_df = pd.DataFrame(ford_app_ap_reviews)

In [5]:
joined_df = [img_reviews_df, us_reviews_df, eu_reviews_df, ap_reviews_df]
all_reviews = pd.concat(joined_df, ignore_index = True)
avg_rating = all_reviews['Rating'].mean()
all_reviews_copy = all_reviews.copy()

In [6]:
all_reviews_copy.drop(labels=None, axis=0, index = 1994, inplace = True)
all_reviews_copy.reset_index(drop=True)

,Name,Rating,Review,Date,Country,Version
0,Karamveer Singh,1,"Hi team , my session logged of and now I'm not...",2022-11-07 13:54:48,ZA,4.22.0
1,Rajesh Chauhan,4,This app does not show the location of the car...,2022-11-07 10:44:35,ZA,4.22.0
2,Vijay Pandey,2,The most useful n the best thing in this app i...,2022-11-05 20:42:47,ZA,4.22.0
3,B Bala,1,Very worst app map will zoom out language auto...,2022-11-05 09:30:48,ZA,4.22.0
4,Vinayak Jadhav,2,After recent update the app is malfunctioning....,2022-11-05 02:37:04,ZA,4.22.0
...,...,...,...,...,...,...
1994,Ajay pandey,1,This app is not useful not show exactly locati...,2021-07-08 15:00:13,AU/NZ,3.25.0
1995,Reya Patel,5,App not working,2021-07-08 13:43:36,AU/NZ,3.25.0
1996,antesh chaudhary,1,👎👎👎👎useless Ford pass app,2021-07-08 09:49:41,AU/NZ,3.25.0
1997,Kiran Dusane,1,"Useless app, Ever not working . Wastage of tim...",2021-07-08 09:21:17,AU/NZ,3.25.0


In [7]:
all_reviews_df1 = all_reviews.copy()

In [8]:
df1_eg = all_reviews.copy()

In [9]:
from datetime import date, timedelta
today = date.today()
yesterday = today - timedelta(1)
all_reviews_df1['Date'] = [d.date() for d in all_reviews_df1['Date']]
yesterday_reviews = all_reviews_df1[all_reviews_df1['Date'] == yesterday]

In [10]:
all_reviews_df1['Platform'] ='Android'

In [11]:
def flair_prediction(x):
    sentences = Sentence(x)
#     Add labels for Zero-shot -Modelling
#     labels = ["positive sentiment", "negative sentiment", "neutral sentiment"]
#     tars.predict_zero_shot(sentences, labels)
    tars.predict(sentences) 
    # print(sentences)
    score = sentences.labels
    if "positive" in str(score):
        return "Positive"
#     if "very_negative" in str(score):
#         return "Very Negative"
    elif "negative" in str(score):
        return "Negative"
    elif "neutral" in str(score):
        return "Neutral"

def flair_prediction_value(x):
    sentence = Sentence(x)
    tars.predict(sentence)
    score = sentence.get_label().score
    return score

In [12]:
df1 = all_reviews_df1.copy()
df1.drop_duplicates(subset="Name", keep='first', inplace=True)
df1.reset_index(drop = True, inplace=True)

In [13]:
df1 = df1[["Name", "Rating", "Review", "Date","Country","Version","Platform"]]
df1["word_count"] = df1['Review'].str.split().str.len()
df1['count'] = 1
df1 = df1[df1["word_count"] > 0]
df1 = df1.groupby("Rating").sample(n=1000, random_state=1, replace=True).reset_index()
df1
# sia = TextClassifier.load('en-sentiment')
tars = TARSClassifier.load('tars-base')
start = time.time()
df1["sentiment"] = df1["Review"].apply(flair_prediction)
end = time.time()
print(end - start)

2022-11-09 00:39:52,610 loading file /Users/pvani/.flair/models/tars-base-v8.pt
1449.014263868332


In [14]:
df1.drop_duplicates(subset="Name", keep='first', inplace=True)
df1.reset_index(drop = True, inplace=True)

In [15]:
df1["sentiment_score_mean"] = df1["Review"].apply(flair_prediction_value)

In [16]:
df1["sentences"] = df1["Review"].apply(tokenize.sent_tokenize)
start = time.time()
df1["sentiment_score"] = df1["sentences"].apply(lambda x: [flair_prediction(y) for y in x])
end = time.time()
print(end - start)
df1["sentiment_one"] = df1["sentiment_score"].apply(lambda x: max(set(x), key=x.count))

728.3865852355957


In [17]:
df2= df1.sort_values('Date',ascending = False)
# df2

In [18]:
df_grouped_2 = df1.groupby(["Rating", "sentiment"]).sum()
# df_grouped_2

/var/folders/dq/__y320sd17v2m6y4l6m7cwp1px9brc/T/ipykernel_78260/3392540313.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped_2 = df1.groupby(["Rating", "sentiment"]).sum()


In [19]:
positive_reviews = df2[(df2['sentiment'] == "Positive") & (df2['sentiment_score_mean'] > 0.985066)].copy()
positive_reviews.reset_index(drop=True, inplace=True)
# positive_reviews

In [20]:
negative_reviews = df2[(df2['sentiment'] == "Negative") & (df2['sentiment_score_mean'] > 0.95)].copy()
negative_reviews.reset_index(drop=True, inplace=True)
# negative_reviews

In [21]:
mixed_reviews_1 = df2[(df2['sentiment'] == "Negative") & (df2['sentiment_score_mean'] < 0.60)].copy()
mixed_reviews_2 = df2[((df2['sentiment'] == "Positive") & (df2['sentiment_score_mean'] < 0.99))].copy()
mixed_joint = [mixed_reviews_1, mixed_reviews_2]
mixed_reviews = pd.concat(mixed_joint, ignore_index = True)
mixed_reviews.reset_index(drop=True, inplace=True)
mixed_reviews['sentiment'] = "Mixed"

In [22]:
neutral_reviews = df2[(df2['sentiment'] == "Neutral") & (df2['sentiment_score_mean'] > 0.80)].copy()
neutral_reviews.reset_index(drop=True, inplace=True)
# neutral_reviews

In [23]:
positiveReviews = positive_reviews[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
negativeReviews = negative_reviews[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
mixedReviews = mixed_reviews[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
neutralReviews = neutral_reviews[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
neutralReviews

,Name,Rating,Review,Country,Date,sentiment,sentiment_score_mean,Version,Platform
0,Daren Lucas,3,Fr%^%,NA,2022-11-05,Neutral,0.838956,4.20.0,Android
1,Sean S.,4,It's ok. Still some bugs to work out such as s...,NA,2022-11-05,Neutral,0.970849,4.22.0,Android
2,Joseph Cavesina,4,The app has gotten much better over the years....,NA,2022-11-04,Neutral,0.987845,4.22.0,Android
3,mizztink85,4,I'm having a lot issues when I start my car it...,NA,2022-11-03,Neutral,0.969846,4.22.0,Android
4,Lance Lucas,4,Always nice to arrive to a warm vehicle when i...,NA,2022-11-02,Neutral,0.835033,4.22.0,Android
...,...,...,...,...,...,...,...,...,...
76,Sonujohal17@gmail.com Johal,2,Okok,ZA,2021-09-01,Neutral,0.951064,3.28.0,Android
77,ashish chikate,1,App colapses when I check vehicle details.,ZA,2021-08-31,Neutral,0.926468,3.28.0,Android
78,Anand Raj Essar,1,Very basic features..,ZA,2021-08-28,Neutral,0.907744,3.28.0,Android
79,Gajju Bhai,5,Ok,ZA,2021-08-10,Neutral,0.923344,3.27.0,Android


In [24]:
differentiated_df = [positiveReviews,negativeReviews,mixed_reviews,neutralReviews]
diff_df = pd.concat(differentiated_df, ignore_index = True)
diff_df.reset_index(drop=True, inplace=True)

In [25]:
def clean_text(text):
    content_texts = text
    contents_string=str(content_texts)
    contents_string_cleaned=re.sub(r'[^a-z A-Z]',' ',contents_string)
    contents_string_cleaned=re.sub(r'\b\w{1,2}\b',' ',contents_string_cleaned)
    contents_string_cleaned=re.sub(r' +',' ',contents_string_cleaned)
    contents_string_cleaned=contents_string_cleaned.lower()
#addedd below will remove later
    contents_string_cleaned = "".join([word for word in contents_string_cleaned])
    doc = nlp_model(contents_string_cleaned)
    contents_string_cleaned = [word.lemma_ for word in doc]
    contents_string_cleaned = [word for word in contents_string_cleaned if word not in nlp_model.Defaults.stop_words]
    contents_string_cleaned = " ".join([word for word in contents_string_cleaned])
    # return text
    
    return contents_string_cleaned

In [26]:
dfDash = diff_df[(diff_df['sentiment']== 'Positive') | (df1['sentiment'] == 'Negative')].copy()
dfDash[dfDash.Review.map(lambda x: x.isascii())]

/var/folders/dq/__y320sd17v2m6y4l6m7cwp1px9brc/T/ipykernel_78260/2854941041.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfDash = diff_df[(diff_df['sentiment']== 'Positive') | (df1['sentiment'] == 'Negative')].copy()


,Name,Rating,Review,Country,Date,sentiment,sentiment_score_mean,Version,Platform,index,word_count,count,sentences,sentiment_score,sentiment_one
0,J Matt Buedel,5,"Great app Easy to use, always starts up withou...",NA,2022-11-02,Positive,0.999903,4.22.0,Android,NaN,NaN,NaN,NaN,NaN,NaN
1,Michelle Olguin,5,Love this app,NA,2022-10-30,Positive,0.994827,4.22.0,Android,NaN,NaN,NaN,NaN,NaN,NaN
2,Louise Melhuish,5,Excellent,NA,2022-10-28,Positive,0.992832,4.17.0,Android,NaN,NaN,NaN,NaN,NaN,NaN
3,Catherine Harvey,5,I truly love this system. Have never had any i...,NA,2022-10-27,Positive,0.997723,None,Android,NaN,NaN,NaN,NaN,NaN,NaN
4,Joseph Graves,5,Great,NA,2022-10-26,Positive,0.987412,4.22.0,Android,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,Funny Irony,4,Updated version is much better,ZA,2021-12-19,Neutral,0.848826,None,Android,NaN,NaN,NaN,NaN,NaN,NaN
866,ahire hemant,5,working fine,ZA,2021-11-27,Neutral,0.861058,3.35.0,Android,NaN,NaN,NaN,NaN,NaN,NaN
870,Sonujohal17@gmail.com Johal,2,Okok,ZA,2021-09-01,Neutral,0.951064,3.28.0,Android,NaN,NaN,NaN,NaN,NaN,NaN
871,ashish chikate,1,App colapses when I check vehicle details.,ZA,2021-08-31,Neutral,0.926468,3.28.0,Android,NaN,NaN,NaN,NaN,NaN,NaN


iOS BEGIN

In [27]:
#fetching US Reviews
df_us = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/us/rss/customerreviews/page='+str(i)+'/id=1095418609/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry = feed['entry']
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_us = pd.concat([df_us,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)
    

In [28]:
#fetching NZ Reviews
df_nz = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/nz/rss/customerreviews/page='+str(i)+'/id=1413892033/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry=feed["entry"]
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_nz = pd.concat([df_nz,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)

Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist


In [29]:
#fetching UK Reviews

df_uk = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/gb/rss/customerreviews/page='+str(i)+'/id=1141464928/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry = feed['entry']
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_uk = pd.concat([df_uk,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)

In [30]:
#fetching Mexico Reviews

df_mx = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/mx/rss/customerreviews/page='+str(i)+'/id=1095418609/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry = feed['entry']
            print(entry)
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_mx = pd.concat([df_mx,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)

[{'author': {'uri': {'label': 'https://itunes.apple.com/mx/reviews/id430543674'}, 'name': {'label': 'jesus.gart'}, 'label': ''}, 'updated': {'label': '2022-11-06T07:49:10-07:00'}, 'im:rating': {'label': '4'}, 'im:version': {'label': '4.22.1'}, 'id': {'label': '9261739633'}, 'title': {'label': 'Localizador'}, 'content': {'label': 'Quiciera tener la posibilidad de saber la velocidad a la que viaja el vehículo', 'attributes': {'type': 'text'}}, 'link': {'attributes': {'rel': 'related', 'href': 'https://itunes.apple.com/mx/review?id=1095418609&type=Purple%20Software'}}, 'im:voteSum': {'label': '0'}, 'im:contentType': {'attributes': {'term': 'Application', 'label': 'Aplicación'}}, 'im:voteCount': {'label': '0'}}, {'author': {'uri': {'label': 'https://itunes.apple.com/mx/reviews/id274488606'}, 'name': {'label': 'Neito1955'}, 'label': ''}, 'updated': {'label': '2022-10-22T13:10:06-07:00'}, 'im:rating': {'label': '5'}, 'im:version': {'label': '4.22.0'}, 'id': {'label': '9211591074'}, 'title': 

In [31]:
#fetching South Africa Reviews

df_za = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/za/rss/customerreviews/page='+str(i)+'/id=1548761721/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry = feed['entry']
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_za = pd.concat([df_za,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)

Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist


In [32]:
#fetching Australia Reviews

df_au = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/au/rss/customerreviews/page='+str(i)+'/id=1413892033/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry = feed['entry']
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_au = pd.concat([df_au,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)

Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist


In [33]:
#fetching India Reviews

df_in = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/in/rss/customerreviews/page='+str(i)+'/id=1462652145/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry = feed['entry']
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_in = pd.concat([df_in,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)

Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist
Page doesn't exist


In [34]:
#fetching Canada Reviews

df_ca = pd.DataFrame()
for i in range(1,11):
    response_API = requests.get('https://itunes.apple.com/ca/rss/customerreviews/page='+str(i)+'/id=1095418609/sortBy=mostRecent/json')
    if response_API.status_code == 200:
        try:
            data = response_API.json()
            feed = data['feed']
            entry = feed['entry']
            df1 = pd.json_normalize(entry)
            df1['Name'] = df1['author.name.label']
            df1['Date'] = df1['updated.label']
            df1['Rating'] = df1['im:rating.label']
            df1['Review'] = df1['content.label']
            df1['Version'] = df1['im:version.label']
            df1 = df1[['Name','Date','Rating','Review','Version']]
            df_ca = pd.concat([df_ca,df1])
        except:
            print("Page doesn't exist")
    else:
        print("Error fetching on page "+i)

In [35]:
df_us['Country'] ='US'
df_uk['Country'] ='UK'
df_mx['Country']='MX'
df_za['Country']='ZA'
df_au['Country']='AU'
df_ca['Country']='CA'
df_nz['Country']='NZ'

In [36]:
combine_df = [df_us, df_uk, df_mx, df_za, df_au, df_ca, df_nz]
all_reviews_df_iOS = pd.concat(combine_df, ignore_index = True)

In [37]:
from dateutil import parser
all_reviews_df_iOS['Date'] = [parser.parse(i) for i in all_reviews_df_iOS['Date']]
all_reviews_df_iOS['Date'] =[i.date() for i in all_reviews_df_iOS['Date']]

In [38]:
all_reviews_df_iOS['Platform'] = 'iOS'
all_reviews_df_iOS

,Name,Date,Rating,Review,Version,Country,Platform
0,johnSsymour,2022-11-07,1,Won’t lock or unlock doors. Won’t start my bra...,4.22.1,US,iOS
1,ygbbhtruion,2022-11-07,1,When I purchased my 2020 F150 it worked great....,4.22.1,US,iOS
2,Lucciani22,2022-11-06,1,I have tried several times to get help w my 20...,4.22.1,US,iOS
3,81 brandy,2022-11-06,1,My remote start worked great until I switched ...,4.22.1,US,iOS
4,tbz20098789,2022-11-06,1,Doesn’t work half the time,4.22.1,US,iOS
...,...,...,...,...,...,...,...
1841,LHEverest,2020-10-02,2,App connects to the truck ok but all the good ...,3.8.0,NZ,iOS
1842,10hendrix,2020-09-26,1,Have tired endlessly to sync with my ford rang...,3.8.0,NZ,iOS
1843,fatkat.68,2020-08-25,1,app won’t go past terms & condition my XR afte...,3.5.0,NZ,iOS
1844,Whoever1a,2020-08-18,1,Good app with my connected vehicle,3.5.0,NZ,iOS


In [39]:
df1_ios = all_reviews_df_iOS.copy()
df1_ios.drop_duplicates(subset="Name", keep='first', inplace=True)
df1_ios.reset_index(drop = True, inplace=True)

In [40]:
df1_ios = df1_ios[["Name", "Rating", "Review", "Date","Country","Version","Platform"]]
df1_ios["word_count"] = df1_ios['Review'].str.split().str.len()
df1_ios['count'] = 1
df1_ios = df1_ios[df1_ios["word_count"] > 0]
df1_ios = df1_ios.groupby("Rating").sample(n=1000, random_state=1, replace=True).reset_index()
df1_ios
# sia = TextClassifier.load('en-sentiment')
tars = TARSClassifier.load('tars-base')
start = time.time()
df1_ios["sentiment"] = df1_ios["Review"].apply(flair_prediction)
end = time.time()
print(end - start)

2022-11-09 01:22:52,936 loading file /Users/pvani/.flair/models/tars-base-v8.pt
1609.296392917633


In [41]:
df1_ios.drop_duplicates(subset="Name", keep='first', inplace=True)
df1_ios.reset_index(drop = True, inplace=True)

In [42]:
df1_ios["sentiment_score_mean"] = df1_ios["Review"].apply(flair_prediction_value)

In [43]:
df1_ios["sentences"] = df1_ios["Review"].apply(tokenize.sent_tokenize)
start = time.time()
df1_ios["sentiment_score"] = df1_ios["sentences"].apply(lambda x: [flair_prediction(y) for y in x])
end = time.time()
print(end - start)
df1_ios["sentiment_one"] = df1_ios["sentiment_score"].apply(lambda x: max(set(x), key=x.count))

1334.638018131256


In [44]:
df2_ios= df1_ios.sort_values('Date',ascending = False)
df2_ios
df_grouped_2_ios = df1_ios.groupby(["Rating", "sentiment"]).sum()
df_grouped_2_ios

/var/folders/dq/__y320sd17v2m6y4l6m7cwp1px9brc/T/ipykernel_78260/2756019071.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped_2_ios = df1_ios.groupby(["Rating", "sentiment"]).sum()


index  word_count  count  sentiment_score_mean
Rating sentiment                                                 
1      Negative   518819       19357    560            506.662343
       Neutral     16102         342     16             12.709071
       Positive    22358         435     21             18.329136
2      Negative   219754       11112    236            205.806628
       Neutral     27581         892     23             19.495876
       Positive    12601         258     12             10.781860
3      Negative   165114        6634    164            144.416277
       Neutral     35494        1327     40             34.301782
       Positive    18589         454     18             15.675420
4      Negative    47387        1930     51             43.710133
       Neutral     42151        1862     39             33.600980
       Positive    26909         875     29             26.496808
5      Negative    25107         957     29             25.363261
       Neutral     24533         817     31             24.071409
       Positive   138131        3346    188            180.120920

In [45]:
positive_reviews_ios = df2_ios[(df2_ios['sentiment'] == "Positive") & (df2_ios['sentiment_score_mean'] > 0.985066)].copy()
positive_reviews_ios.reset_index(drop=True, inplace=True)
negative_reviews_ios = df2_ios[(df2_ios['sentiment'] == "Negative") & (df2_ios['sentiment_score_mean'] > 0.95)].copy()
negative_reviews_ios.reset_index(drop=True, inplace=True)
mixed_reviews_1_ios = df2_ios[(df2_ios['sentiment'] == "Negative") & (df2_ios['sentiment_score_mean'] < 0.60)].copy()
mixed_reviews_2_ios = df2_ios[((df2_ios['sentiment'] == "Positive") & (df2_ios['sentiment_score_mean'] < 0.99))].copy()
mixed_joint_ios = [mixed_reviews_1_ios, mixed_reviews_2_ios]
mixed_reviews_ios = pd.concat(mixed_joint_ios, ignore_index = True)
mixed_reviews_ios.reset_index(drop=True, inplace=True)
mixed_reviews_ios['sentiment'] = "Mixed"
neutral_reviews_ios = df2_ios[(df2_ios['sentiment'] == "Neutral") & (df2_ios['sentiment_score_mean'] > 0.80)].copy()
neutral_reviews_ios.reset_index(drop=True, inplace=True)
# neutral_reviews

In [46]:
positiveReviews_ios = positive_reviews_ios[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
negativeReviews_ios = negative_reviews_ios[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
mixedReviews_ios = mixed_reviews_ios[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
neutralReviews_ios = neutral_reviews_ios[["Name", "Rating", "Review","Country","Date","sentiment","sentiment_score_mean","Version","Platform"]]
# neutralReviews_ios

In [47]:
differentiated_df_ios = [positiveReviews_ios,negativeReviews_ios,mixed_reviews_ios,neutralReviews_ios]
diff_df_ios = pd.concat(differentiated_df_ios, ignore_index = True)
diff_df_ios.reset_index(drop=True, inplace=True)
# diff_df_ios

In [48]:
dfDash_ios = diff_df_ios[(diff_df_ios['sentiment']== 'Positive') | (df1_ios['sentiment'] == 'Negative')].copy()
dfDash_ios[dfDash_ios.Review.map(lambda x: x.isascii())]

/var/folders/dq/__y320sd17v2m6y4l6m7cwp1px9brc/T/ipykernel_78260/3508880248.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfDash_ios = diff_df_ios[(diff_df_ios['sentiment']== 'Positive') | (df1_ios['sentiment'] == 'Negative')].copy()


,Name,Rating,Review,Country,Date,sentiment,sentiment_score_mean,Version,Platform,index,word_count,count,sentences,sentiment_score,sentiment_one
0,bryce 45,5,Works really well. No issues.,CA,2022-11-06,Positive,0.991696,4.22.1,iOS,NaN,NaN,NaN,NaN,NaN,NaN
1,gr8wyteyeti93,5,Awesome,US,2022-11-05,Positive,0.993755,4.22.1,iOS,NaN,NaN,NaN,NaN,NaN,NaN
2,Wookash. O,5,"Best, most efficient truck friendly app on the...",US,2022-11-05,Positive,0.998619,4.22.1,iOS,NaN,NaN,NaN,NaN,NaN,NaN
3,Sirr Cat,5,So far has been great I love what I can do wit...,US,2022-11-03,Positive,0.986822,4.22.1,iOS,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Drawbaugh,5,This app has helped me out a lot especially wh...,US,2022-11-03,Positive,0.994959,4.22.1,iOS,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,Eddy2794,3,I enjoy what the app has to offer and some of ...,UK,2021-08-17,Neutral,0.939031,3.27.0,iOS,NaN,NaN,NaN,NaN,NaN,NaN
869,Just21!!!,1,The guard mode sends alerts at random times in...,UK,2021-07-08,Neutral,0.974261,3.25.0,iOS,NaN,NaN,NaN,NaN,NaN,NaN
872,laserman1234,3,Good app but the time it takes for the vehicle...,NZ,2021-05-29,Neutral,0.945536,3.23.0,iOS,NaN,NaN,NaN,NaN,NaN,NaN
873,mickeydawes,3,Most things work ok but recorded journeys are ...,UK,2021-05-11,Neutral,0.884592,3.21.0,iOS,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
both_platform=[dfDash,dfDash_ios]
df1_final = pd.concat(both_platform, ignore_index = True)
df1_final.drop_duplicates(subset="Name", keep='first', inplace=True)
df1_final.reset_index(drop = True, inplace=True)

In [58]:
df1_final.to_csv("finalDF.csv")

In [50]:
pos_combine =[positiveReviews,positiveReviews_ios]
pos_final = pd.concat(pos_combine, ignore_index = True)
pos_final.drop_duplicates(subset="Name", keep='first', inplace=True)
pos_final.reset_index(drop = True, inplace=True)

neg_combine =[negativeReviews,negativeReviews_ios]
neg_final = pd.concat(neg_combine, ignore_index = True)
neg_final.drop_duplicates(subset="Name", keep='first', inplace=True)
neg_final.reset_index(drop = True, inplace=True)

neu_combine =[neutralReviews,neutralReviews_ios]
neu_final = pd.concat(neu_combine, ignore_index = True)
neu_final.drop_duplicates(subset="Name", keep='first', inplace=True)
neu_final.reset_index(drop = True, inplace=True)

mixed_combine =[mixedReviews,mixedReviews_ios]
mix_final = pd.concat(mixed_combine, ignore_index = True)
mix_final.drop_duplicates(subset="Name", keep='first', inplace=True)
mix_final.reset_index(drop = True, inplace=True)

In [55]:
posCSV = pos_final[["Name","Review","Date","Country","Version","Rating","Platform"]]
negCSV = neg_final[["Name","Review","Date","Country","Version","Rating","Platform"]]
neuCSV = neu_final[["Name","Review","Date","Country","Version","Rating","Platform"]]
mixCSV = mix_final[["Name","Review","Date","Country","Version","Rating","Platform"]]

In [57]:
posCSV.to_csv("PositiveFeedback.csv")
negCSV.to_csv("NegativeFeedback.csv")
neuCSV.to_csv("NeutralFeedback.csv")
mixCSV.to_csv("MixedFeedback.csv")